In [1]:
import pandas as pd
import re

# KR operations

In [2]:
def extract_locality_and_street(input_string):
    # SUbstring prefixes before locality name
    prefixes = ["г.", "г ", "с.", "с ", "кп ", "к.п.", "Город ", "город ", "п.", "п "]
    locality_keyword = ":"

    locality = ""
    street = input_string

    locality_index = input_string.find(locality_keyword)
    input_string = input_string[locality_index:]

    # Substring search
    for prefix in prefixes:
        index = input_string.rfind(prefix)
        if index != -1:
            comma_index = input_string.find(',', index)
            locality = input_string[index:comma_index].strip()
            street_start = comma_index + 1
            street = input_string[street_start:].strip()
            break

    return locality, street

In [3]:
KR_files = ["KR on 01.01.2017 unprotected.xlsx", "KR on 01.01.2018 unprotected.xlsx", "KR on 01.01.2019 unprotected.xlsx"]

results = []

for KR_file in KR_files:
    filexlsx = pd.read_excel("train_data/KC_TR_KR/"+KR_file, skiprows=5) # Skip header

    columns_to_drop = [0, 2, 4, 6, 7, 10]
    filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)

    new_column_names = ["name", "estimated_cost", "approved_year_start", "used_year_end", "expected_year_end"]
    filexlsx.columns = new_column_names

    target_line_startswith = ["Выборочный", "Капитальный", "Новый", "Ремонт"]
    filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]

    filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

    results.append(filtered_rows)

/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_36407/3769734642.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_36407/3769734642.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders

In [4]:
# 2016
results[0]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
5,Капитальный ремонт подземного сооружения (Лит....,4088.43,4077.25,3927.48,3927.48,г. Владивосток,"ул. Светланская, д. 71"
8,"Капитальный ремонт здания РКЦ Находка, располо...",19001.91,1149.83,1143.30,1143.30,г. Находка,"проспект Находкинский, 13"
13,Капитальный ремонт гаража по адресу: Амурская ...,13677.35,12858.01,12858.01,12858.01,г. Благовещенск,"пер. Святителя Иннокентия, 17"
16,Выборочный капитальный ремонт административног...,18034.12,1298.34,1298.33,1298.33,г. Тында,"ул. Профсоюзная, 12"
19,Выборочный капитальный ремонт административног...,448.72,478.98,448.72,448.72,г. Благовещенск,"пер. Св. Иннокентия, 17"
24,Выборочный капитальный ремонт здания гаража на...,4439.86,199.79,199.79,199.79,г. Петропавловск-Камчатский,"ул. Ломоносова, 2"
29,Выборочный капитальный ремонт сетей теплоснабж...,97.37,97.37,97.37,97.37,г. Магадан,"ул. Пролетарская, д. 8, строение 1, литера А, ..."
34,Выборочный капитальный ремонт административног...,2344.49,3281.74,2344.49,2344.49,г. Покровск,"ул. Орджоникидзе, д. 24"
39,"Выборочный капитальный ремонт ""Сооружения комм...",184.94,184.95,184.94,184.94,,"Выборочный капитальный ремонт ""Сооружения комм..."
42,Выборочный капитальный ремонт в части усиления...,3634.45,3621.51,3308.46,3308.46,г. Южно-Сахалинск,"Коммунистический проспект, 47 (литер А)"


In [5]:
# 2017
results[1]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
5,Капитальный ремонт помещений амбулатории и кро...,10943.56,670,670.00,670.00,г. Владивосток,"ул. Светланская, 71"
8,"Капитальный ремонт здания РКЦ Находка, располо...",21443.21,8817.68,8817.67,8817.67,г. Находка,"проспект Находкинский, д. 13"
11,Выборочный капитальный ремонт ограждения и вор...,5349.85,5377.36,5349.85,5349.85,г. Уссурийск,"ул. Некрасова, 102"
14,Капитальный ремонт водопровода в гараже на 10 ...,283.66,283.66,283.66,283.66,г. Уссурийск,"ул. Некрасова, д. 102"
17,Новый объект. Выборочный капитальный ремонт п...,154.41,154.42,154.41,154.41,г. Уссурийск,"ул. Некрасова, 102"
22,Выборочный капитальный ремонт помещений 3 этаж...,19875.80,21461.5,19875.80,19875.80,г. Хабаровск,"ул. Муравьева-Амурского, 21"
25,Выборочный капитальный ремонт колодца ВК1 объе...,176.18,196.5,176.18,176.18,г. Хабаровск,"ул. Калинина, 156"
30,Капитальный ремонт кровли блока А здания по ад...,2506.52,2506.52,2506.52,2506.52,г. Петропавловск-Камчатский,"просп. К. Маркса, 29/2"
35,Выборочный капитальный ремонт первого этажа во...,2999.90,2999.9,2999.90,2999.90,г. Южно-Сахалинск,"Коммунистический проспект, 47"
38,"Выборочный капитальный ремонт объекта ""Сооруже...",1699.56,1699.56,849.78,849.78,,"Выборочный капитальный ремонт объекта ""Сооруже..."


In [6]:
# 2018
results[2]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
5,Капитальный ремонт помещений и кровли в пятиэт...,670.00,9170.91,9170.90,12.2018,г. Владивосток,"ул. Светланская, 71"
8,"Капитальный ремонт здания РКЦ Находка, располо...",9960.98,11482.32,11482.32,5.2018,г.Находка,"проспект Находкинский, д. 13"
11,"Капитальный ремонт здания, расположенного по а...",NaN,375.72,375.71,10.2019,г.Владивосток,"ул. Калинина, 261"
14,Капитальный ремонт внешнего электроснабжения з...,NaN,290,290.00,6.2020,г. Владивосток,"ул. Светланская, д.71, д.73, д.73а"
17,"Капитальный ремонт нежилых помещений (склада),...",NaN,226.46,226.45,9.2018,г.Владивосток,"ул.Фонтанная, 19"
20,Капитальный ремонт инженерных сетей на объекта...,NaN,1227.79,1227.79,7.2018,г. Владивосток,"ул. Пушкинская, 8б"
25,Выборочный капитальный ремонт помещений 2 эта...,NaN,156.65,0.00,8.2019,г. Хабаровск,"ул. Муравьева-Амурского,21"
28,Выборочный капитальный ремонт аварийных участк...,NaN,175.56,140.46,12.2018,г. Хабаровск,"ул. Воронежская, дом 154-А (реестровый №73-04-..."
33,Выборочный капитальный ремонт здания гаража на...,199.79,3397.65,3397.65,8.2018,г. Петропавловск-Камчатский,"ул. Ломоносова, 2"
36,Выборочный капитальный ремонт системы хозяйств...,NaN,490.53,490.53,8.2018,г. Петропавловск-Камчатский,"просп. К.Маркса, 29/2"


## New version sheets

In [7]:
KR_files = ["KR on 01.01.2020.xlsx", "KR on 01.01.2021.xlsx", "KR on 01.01.2022.xlsx"]

for KR_file in KR_files:
    filexlsx = pd.read_excel("train_data/KC_TR_KR/"+KR_file, skiprows=2) # Skip header

    columns_to_drop = [0, 2, 4, 5, 7, 9, 10, 12, 13, 14, 15, 16, 17]
    filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)

    new_column_names = ["name", "estimated_cost", "approved_year_start", "used_year_end", "expected_year_end"]
    filexlsx.columns = new_column_names

    ### Old version sheets were in thousands roubles 
    columns_to_divide = ['estimated_cost', 'approved_year_start', 'used_year_end', 'expected_year_end']
    # Convert the specified columns to numeric type
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].apply(pd.to_numeric, errors='coerce')
    # Divide
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].div(1000)
    # Round the numeric values to two decimal places
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].round(2)
    # Convert the columns back to string type
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].astype(str)

    target_line_startswith = ["Выборочный", "Капитальный", "Новый", "Ремонт"]
    filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]

    filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

    results.append(filtered_rows)


/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_36407/4001036180.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_36407/4001036180.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders

In [8]:
# 2019
results[3]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Капитальный ремонт здания убежища (лит. А) Дал...,9437.31,1800.13,0.0,0.0,г. Владивосток,"ул. Семеновская, 29а"
14,Капитальный ремонт внешнего электроснабжения з...,6706.61,6416.61,6416.61,6416.61,г. Владивосток,"ул. Светланская, д.71, д.73, д.73а"
17,"Капитальный ремонт здания, расположенного по а...",83550.9,1469.48,1469.48,1469.48,г. Владивосток,"ул. Светланская, 71"
20,Капитальный ремонт нежилых помещений и помещен...,30232.72,1653.67,0.0,0.0,г. Владивосток,"ул. Светланская, д.73"
23,Выборочный капитальный ремонт помещения бокс-...,12529.12,1429.23,0.0,0.0,г. Владивосток,"ул. Светланская, 71"
26,"Капитальный ремонт здания, расположенного по а...",4529.89,1653.94,1654.43,1654.43,с. Вольно-Надеждинское,"ул. Пушкина, д. 59"
29,"Выборочный капитальный ремонт РКЦ, пристройки ...",15492.99,14736.19,0.0,0.0,г. Уссурийск,"ул. Некрасова, д.102"
32,Выборочный капитальный ремонт гаража РКЦ на 2 ...,2014.98,1877.47,908.62,908.62,г. Уссурийск,"ул. Некрасова, д.102"
35,Выборочный капитальный ремонт нежилых помещен...,2875.63,263.6,0.0,0.0,г. Владивосток,"ул. Светланская, д.73"
38,Выборочный капитальный ремонт здания Централь...,3942.55,258.0,258.0,258.0,г. Владивосток,"ул. Океанский проспект, д.34"


In [9]:
# 2020
results[4]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Выборочный капитальный ремонт здания Централь...,3942.55,1836.16,0.0,0.0,г. Владивосток,"ул. Океанский проспект, д.34"
14,Выборочный капитальный ремонт здания Централь...,97.78,97.78,97.78,97.78,г. Владивосток,"Океанский проспект, д.34 в части замены дверно..."
17,Выборочный капитальный ремонт нежилых помещен...,2907.5,2907.5,235.59,235.59,г. Владивосток,"ул. Светланская, д.73"
20,Выборочный капитальный ремонт помещения бокс-...,12784.08,12309.38,2308.49,2308.49,г. Владивосток,"ул. Светланская, 71"
23,Капитальный ремонт здания убежища (лит. А) Дал...,9465.23,9163.94,9198.95,9198.95,г. Владивосток,"ул. Семеновская, 29а"
26,Капитальный ремонт нежилых помещений и помещен...,31288.37,30776.85,15126.51,15126.51,г. Владивосток,"ул. Светланская, д.73"
29,"Капитальный ремонт здания, расположенного по а...",4924.75,2981.81,2981.31,2981.31,с. Вольно-Надеждинское,"ул. Пушкина, д. 59"
32,"Выборочный капитальный ремонт РКЦ, пристройки ...",16313.17,15556.36,5325.76,5325.76,г. Уссурийск,"ул. Некрасова, д.102"
35,Выборочный капитальный ремонт гаража РКЦ на 2 ...,1946.33,895.2,895.2,895.2,г. Уссурийск,"ул. Некрасова, д.102"
38,Выборочный капитальный ремонт гаража РКЦ на 10...,1534.44,1483.53,1483.53,1483.53,г. Уссурийск,"ул. Некрасова, д.102"


In [10]:
# 2021
results[5]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Выборочный капитальный ремонт здания Централь...,4415.99,4157.99,3871.0,3871.0,г. Владивосток,"ул. Океанский проспект, д.34"
14,Выборочный капитальный ремонт нежилого здания ...,4243.68,4243.68,139.45,139.45,г. Владивосток,"пр-кт Океанский, д. 34 в части замены лифтовог..."
17,Выборочный капитальный ремонт помещения бокс-...,12475.7,9692.51,3909.53,3909.53,г. Владивосток,"ул. Светланская, 71"
20,Выборочный капитальный ремонт нежилых помещени...,2907.5,2671.9,945.59,945.59,г. Владивосток,"ул. Светланская, д.73"
23,Капитальный ремонт нежилых помещений и помещен...,31288.37,15650.33,11776.82,11776.82,г. Владивосток,"ул. Светланская, д.73"
26,Выборочный капитальный ремонт нежилого здания ...,4074.53,153.69,153.69,153.69,город Уссурийск,"улица Некрасова, здание № 102 в части замены л..."
29,"Выборочный капитальный ремонт РКЦ, пристройки ...",16107.76,1212.79,1212.79,1212.79,г. Уссурийск,"ул. Некрасова, д.102"
32,Выборочный капитальный ремонт с созданием авто...,57.28,57.28,57.28,57.28,г. Владивосток,"пр-кт Океанский, д. 34"
35,Выборочный капитальный ремонт помещения № 14 в...,1212.9,1212.9,1212.9,1212.9,г. Владивосток,"ул. Калинина, д. 261"
39,Капитальный ремонт с перепланировкой помещений...,43837.95,999.62,999.62,999.62,г город Благовещенск,"Город Благовещенск, Переулок Св. Иннокентия, Д..."


# KS operations

In [11]:
KS_files = ["KS on 01.01.2017.xls", "KS on 01.01.2018.xls", "KS on 01.01.2019.xls"]

results_KS = []

for KS_file in KS_files:
    filexlsx = pd.read_excel("train_data/KC_TR_KR/"+KS_file, skiprows=4) # Skip header

    columns_to_drop = [0, 1, 3, 4, 6, 7, 9, 10, 12, 14, 15, 16, 17]
    filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)
    
    new_column_names = ["name", "estimated_cost", "approved_year_start", "used_year_end", "expected_year_end"]
    filexlsx.columns = new_column_names

    target_line_startswith = ["Реконструкция", "Строительство", "Создание", "Приобретение"]
    filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]
    
    filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

    results_KS.append(filtered_rows)

/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_36407/749048099.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_36407/749048099.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/h

In [12]:
# 2016
results_KS[0]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
9,"Реконструкция объектов ""Функциональное помеще...",29266.71,16373.8,15279.96,15279.96,г.Хабаровск,"ул. Муравьева-Амурского, 21 (разделение инжене..."
13,Реконструкция здания Расчетно-кассового центра...,1778.91,1662.96,1579.42,1579.42,п. Ключи,"ул. Кирова, 138"
19,Строительство Административного здания №1 Наци...,333748.33,5217.98,3365,3365,г. Якутск,"ул. Кирова,17"
22,Реконструкция здания Расчетно-кассового центра...,196.79,196.79,103.56,103.56,с. Чурапча,"ул. Ленина, 37 в части системы хозяйственно-пи..."
26,"Реконструкция объекта ""Сооружение ливневая...",10945.44,1560.61,1311.61,1311.61,с. Паратунка,"пансионат ""Светлячок"""
29,"Реконструкция объекта ""Сооружение скважина...",22388.30,2721.46,2121.04,2121.04,с Паратунка,"пансионат ""Светлячок"""
37,Создание системы приточно-вытяжной вентиляции ...,1709.66,1400.31,1400.31,1400.31,г.Тында,"ул. Профсоюзная,12"


In [13]:
# 2017
results_KS[1]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
9,"Реконструкция объектов ""Функциональное помеще...",27505.02,6877.18,6877.17,6877.17,г.Хабаровск,"ул. Муравьева-Амурского, 21 (разделение инжене..."
13,"Реконструкция административного здания, в част...",1832.05,1751.42,1716.39,1716.39,,"Реконструкция административного здания, в част..."
17,Строительство тепловой сети к зданию Расчетно-...,3367.16,3055.43,2968.25,2968.25,с. Тигиль,"ул. Партизанская, 11"
21,Строительство здания спортивного зала в пансио...,93408.50,655.37,NaN,NaN,с. Паратунк,": Камчатский край, Елизовский район, с. Паратунка"
24,"Реконструкция объекта: ""Сооружение скважина хо...",28663.48,530.26,530.26,530.26,с. Паратунка,"пансионат ""Светлячок"""
32,Создание автоматической системы газового пожар...,5251.14,434.59,434.59,434.59,,Создание автоматической системы газового пожар...
36,Создание автоматической системы газового пожар...,3024.36,114.13,99.1,99.1,г. Благовещенск,"пер. Св.Иннокентия, 17"
39,Создание системы приточно-вытяжной вентиляции ...,11644.92,136.03,108.817,108.817,г. Благовещенск,"ул. Б.Хмельницкого, 52/2"
42,Создание системы кондиционирования администрат...,8888.82,126.18,100.94,100.94,г. Благовещенск,"ул. Б. Хмельницкого, 52/2"
45,Создание системы автоматического газового пожа...,3329.38,114.07,56.06,56.06,г. Благовещенск,"ул. Б. Хмельницкого, 52/2"


In [14]:
# 2018
results_KS[2]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
9,Реконструкция внешнего электроснабжения здания...,1573.07,239.47,220.65,220.65,г. Владивосток,"ул. Калинина, д. 261"
13,Реконструкция здания Расчетно-кассового центра...,1167.57,145.92,144.09,144.09,п. Усть-Камчатск,"ул. Ленина, 3"
17,"Реконструкция объекта ""Сооружение ливневая кан...",9634.24,8322.66,8314.5,8314.5,с. Паратунка,"пансионат ""Светлячок"""
25,Создание системы приточно-вытяжной вентиляции ...,963.43,128.33,120,120,г. Уссурийск,"ул. Некрасова, 102"
28,Создание автоматической системы газового пожар...,3166.12,2680.94,2680.94,2680.94,,Создание автоматической системы газового пожар...
31,Создание системы вентиляции в помещениях столо...,3643.39,353.37,330,330,г. Владивосток,"ул. Светланская, д. 73"
34,Создание приточно-вытяжной вентиляции помещени...,534.14,124.91,120,120,,Создание приточно-вытяжной вентиляции помещени...
38,Создание системы кондиционирования воздуха в ...,18338.48,9843.39,9843.39,9843.39,,Создание системы кондиционирования воздуха в ...
41,Создание системы кондиционирования воздуха в с...,7511.06,71.1,71.1,71.1,г.Ленск,"ул. Ленина, д. 70, лит. А, А"
45,Создание системы приточно-вытяжной вентиляции ...,11644.92,27.21,27.2,27.2,г.Благовещенск,"ул. Б. Хмельницкого, 52/2"


## New version sheets

In [15]:
KS_files = ["KS on 01.01.2020.xlsx", "KS on 01.01.2021.xlsx", "KS on 01.01.2022.xlsx"]

for KS_file in KS_files:
    filexlsx = pd.read_excel("train_data/KC_TR_KR/"+KS_file, skiprows=7) # Skip header

    columns_to_drop = [0, 2, 4, 5, 7, 8, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20]
    filexlsx = filexlsx.drop(filexlsx.columns[columns_to_drop], axis=1)

    new_column_names = ["name", "estimated_cost", "approved_year_start", "used_year_end", "expected_year_end"]
    filexlsx.columns = new_column_names

    ### Old version sheets were in thousands roubles 
    columns_to_divide = ['estimated_cost', 'approved_year_start', 'used_year_end', 'expected_year_end']
    # Convert the specified columns to numeric type
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].apply(pd.to_numeric, errors='coerce')
    # Divide
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].div(1000)
    # Round the numeric values to two decimal places
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].round(2)
    # Convert the columns back to string type
    filexlsx[columns_to_divide] = filexlsx[columns_to_divide].astype(str)
    
    target_line_startswith = ["Реконструкция", "Строительство", "Создание", "Приобретение"]
    filtered_rows = filexlsx[filexlsx['name'].apply(lambda x: str(x).startswith(tuple(target_line_startswith)) if pd.notna(x) else False)]

    filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))

    results_KS.append(filtered_rows)

/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_36407/2301092877.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders/hx/c54h6pps2qjgpz_n6c14sk2c0000gp/T/ipykernel_36407/2301092877.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows[['locality', 'street']] = filtered_rows['name'].apply(lambda x: pd.Series(extract_locality_and_street(x)))
/var/folders

In [16]:
# 2019
results_KS[3]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
9,"Приобретение здания по адресу: г. Владивосток,...",169044.57,169044.57,169044.57,169044.57,г. Владивосток,"ул. Светланская, д. 111"
13,Реконструкция теплового пункта Дальневосточног...,667.51,667.51,667.51,667.51,г. Владивосток,"ул. Калинина, 261"
16,Реконструкция теплововго пункта Дальневостчоно...,631.63,631.63,631.63,631.63,г. Владивосток,"ул. Светланская, 73"
23,"Создание ""Автоматизированной Системы Техническ...",2515.26,2036.46,2036.46,2036.46,г. Владивосток,"ул. Светланская, 73, с. Вольно-Надеждинское, у..."
26,Создание приточно-вытяжной вентиляции в здании...,2888.37,685.58,685.57,685.57,с. Вольно-Надеждинское,"ул. Пушкина, д. 59"
29,Создание системы кондиционирования воздуха в п...,19545.15,442.0,442.0,442.0,г. Владивосток,"пр-кт Океанский, 34"
32,Создание системы приточно-вытяжной вентиляции ...,2753.81,2633.81,2633.81,2633.81,г. Уссурийск,"ул. Некрасова, 102"
35,"Создание системы вентиляции в РКЦ, пристройки ...",360.43,360.43,0.0,0.0,г. Уссурийск,"ул. Некрасова, д. 102"
39,Создание системы автоматического газового пожа...,4528.95,4148.04,0.0,0.0,г.Благовещенск,"ул. Б. Хмельницкого, 52/2"
42,Создание системы кондиционирования администрат...,4667.63,4541.45,0.74,0.74,г.Благовещенск,"ул. Б. Хмельницкого, 52/2"


In [17]:
# 2020
results_KS[4]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Реконструкция отдельных помещений информатизац...,38969.03,33446.76,12744.3,12744.3,г. Владивосток,"Океанский проспект, д. 34"
18,Создание автоматической системы газового пожар...,9098.39,8701.19,8701.19,8701.19,с. Вольно-Надеждинское,"ул. Пушкина, д. 59 (лит. А)"
21,Создание приточно-вытяжной вентиляции в здании...,2626.71,1599.68,1599.67,1599.68,с. Вольно-Надеждинское,"ул. Пушкина, д. 59"
24,Создание системы вентиляции в помещениях столо...,9250.36,8920.36,2435.06,2435.06,г. Владивосток,"ул. Светланская, д. 73"
27,Создание системы кондиционирования воздуха в п...,22064.17,16590.86,13423.2,13423.2,г. Владивосток,"пр-кт Океанский, 34"
30,"Создание системы вентиляции в РКЦ, пристройке ...",368.17,368.17,0.0,0.0,г. Уссурийск,"ул. Некрасова, д. 102"
34,Создание системы автоматического газового пожа...,5443.0,4909.3,15.96,15.96,г.Благовещенск,"ул. Б. Хмельницкого, 52/2"
37,Создание системы кондиционирования администрат...,7423.21,5950.88,43.53,43.53,г.Благовещенск,"ул. Б. Хмельницкого, 52/2"
40,Создание системы приточно-вытяжной вентиляции ...,5894.92,4697.87,57.32,57.32,г.Благовещенск,"ул. Б. Хмельницкого, 52/2"
44,Создание автоматической установки газового пож...,9264.73,9098.73,9080.0,9080.0,г.Биробиджан,"проспект 60 -летия образования СССР, 5"


In [18]:
# 2021
results_KS[5]

,name,estimated_cost,approved_year_start,used_year_end,expected_year_end,locality,street
11,Реконструкция отдельных помещений информатизац...,36911.84,23757.67,23757.67,23757.67,г. Владивосток,"Океанский проспект, д. 34"
15,"Реконструкция Здания РКЦ, по адресу: Чукотский...",499.0,499.0,499.0,499.0,г. Билибино,"ул. Ленина, д.6А в части теплового узла"
18,Строительство металлического ограждения здания...,6905.01,132.44,83.51,83.51,г город Благовещенск,"Город Благовещенск, Переулок Св.Иннокентия, До..."
21,Реконструкция газовой автономной котельной Адм...,29830.69,1541.98,1541.98,1541.98,г. Якутск,"ул. Кирова, д. 17"
27,Создание системы кондиционирования воздуха в п...,21740.69,7875.49,7875.49,7875.49,г. Владивосток,"пр-кт Океанский, 34"
30,Создание системы вентиляции в помещениях столо...,8446.86,5681.8,0.0,0.0,г. Владивосток,"ул. Светланская, д. 73"
33,"Создание системы вентиляции в РКЦ, пристройке ...",368.17,368.17,368.17,368.17,г. Уссурийск,"ул. Некрасова, д. 102"
36,Создание автоматической системы газового пожар...,864.09,864.09,864.09,864.09,г. Владивосток,"пр-кт Океанский, д. 34"
40,Создание автоматической системы газового пожар...,9096.52,9096.52,8920.43,8920.43,город Петропавловск-Камчатский,"проспект Карла Маркса, дом № 29/2"
44,Создание автоматизированной системы коммерческ...,2609.98,2609.98,2609.97,2609.97,,Создание автоматизированной системы коммерческ...
